In [6]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [65]:
df = pd.read_parquet("C:\\Users\\Tobi\\Desktop\\New folder\\git\\Energy_Load_Profile_Classification\\Energy_Load_Profile_Classification\\main.parquet")

In [66]:
df = df[['Time','system_kW']]

In [67]:
df['Time'] = pd.to_datetime(df['Time'])

In [68]:
# df['new_date'] = [d.date() for d in df['Time']]
# df['new_time'] = [d.time() for d in df['Time']]

In [69]:
# import pandas as pd

# # Get week number, with pd.DateOffset used to start the week on Sunday
# week_date_values = df["Time"].dt.isocalendar()
# df["week_date"] = (df['Time'] - pd.to_timedelta(df['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# # Combine with "(week n)" values
# df["week_date"] = df["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
# df

In [82]:
# data re-sampled based on an hour
df_by_hr_max = df.resample('H', on='Time').system_kW.max()

df_by_hr_max = pd.DataFrame(df_by_hr_max).reset_index()

df_by_hr_max.rename(columns = {'system_kW':'Max_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_max["Time"].dt.isocalendar()
df_by_hr_max["week_date"] = (df_by_hr_max['Time'] - pd.to_timedelta(df_by_hr_max['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_max["week_date"] = df_by_hr_max["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df_by_hr_max

,Time,Max_Load,week_date
0,2022-09-17 12:00:00,46.666,2022-09-11 (week 37)
1,2022-09-17 13:00:00,51.045,2022-09-11 (week 37)
2,2022-09-17 14:00:00,45.432,2022-09-11 (week 37)
3,2022-09-17 15:00:00,47.489,2022-09-11 (week 37)
4,2022-09-17 16:00:00,59.146,2022-09-11 (week 37)
...,...,...,...
3271,2023-01-31 19:00:00,87.146,2023-01-29 (week 5)
3272,2023-01-31 20:00:00,82.882,2023-01-29 (week 5)
3273,2023-01-31 21:00:00,76.380,2023-01-29 (week 5)
3274,2023-01-31 22:00:00,62.340,2023-01-29 (week 5)


In [83]:
# data re-sampled based on an hour
df_by_hr_min = df.resample('H', on='Time').system_kW.min()

df_by_hr_min = pd.DataFrame(df_by_hr_min).reset_index()

df_by_hr_min.rename(columns = {'system_kW':'Min_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_min["Time"].dt.isocalendar()
df_by_hr_min["week_date"] = (df_by_hr_min['Time'] - pd.to_timedelta(df_by_hr_min['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_min["week_date"] = df_by_hr_min["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df_by_hr_min

,Time,Min_Load,week_date
0,2022-09-17 12:00:00,29.376,2022-09-11 (week 37)
1,2022-09-17 13:00:00,31.066,2022-09-11 (week 37)
2,2022-09-17 14:00:00,35.635,2022-09-11 (week 37)
3,2022-09-17 15:00:00,36.948,2022-09-11 (week 37)
4,2022-09-17 16:00:00,36.043,2022-09-11 (week 37)
...,...,...,...
3271,2023-01-31 19:00:00,72.626,2023-01-29 (week 5)
3272,2023-01-31 20:00:00,70.813,2023-01-29 (week 5)
3273,2023-01-31 21:00:00,57.502,2023-01-29 (week 5)
3274,2023-01-31 22:00:00,52.269,2023-01-29 (week 5)


In [84]:
# data re-sampled based on an hour
df_by_hr_mean = df.resample('H', on='Time').system_kW.mean()

df_by_hr_mean = pd.DataFrame(df_by_hr_mean).reset_index()

df_by_hr_mean.rename(columns = {'system_kW':'Mean_Load'}, inplace = True)

import pandas as pd

# Get week number, with pd.DateOffset used to start the week on Sunday
week_date_values = df_by_hr_mean["Time"].dt.isocalendar()
df_by_hr_mean["week_date"] = (df_by_hr_mean['Time'] - pd.to_timedelta(df_by_hr_mean['Time'].dt.dayofweek, unit='d') + pd.DateOffset(days=-1)).dt.strftime("%Y-%m-%d")

# Combine with "(week n)" values
df_by_hr_mean["week_date"] = df_by_hr_mean["week_date"] + " " + "(week " + week_date_values["week"].astype(str) + ")"
df_by_hr_mean

,Time,Mean_Load,week_date
0,2022-09-17 12:00:00,37.835000,2022-09-11 (week 37)
1,2022-09-17 13:00:00,43.259552,2022-09-11 (week 37)
2,2022-09-17 14:00:00,40.673883,2022-09-11 (week 37)
3,2022-09-17 15:00:00,43.198733,2022-09-11 (week 37)
4,2022-09-17 16:00:00,46.881117,2022-09-11 (week 37)
...,...,...,...
3271,2023-01-31 19:00:00,79.960733,2023-01-29 (week 5)
3272,2023-01-31 20:00:00,77.210667,2023-01-29 (week 5)
3273,2023-01-31 21:00:00,66.348317,2023-01-29 (week 5)
3274,2023-01-31 22:00:00,57.426000,2023-01-29 (week 5)
